In [2]:
import scipy.stats as sps
from scipy.interpolate import interp1d
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from models.CMF_Predictor import CMFRecommender
from tqdm import tqdm
from libmf import mf

In [3]:
def usefulness(c, w, noice):
    # return 1
    # return np.arctan(c - w + noice) / np.pi + 0.5
    return np.sin(c - w + noice) / 2 + 1

In [4]:
customer_distribution = sps.norm(0.6, 0.2)
w_distribution = sps.norm(0, 0.4)

epsilon = sps.norm(0, 0.05)


class Distribution_Handler:
    def __init__(self, distribution):
        self.distribution = distribution

    def rvs(self, size=1):
        if issubclass(type(self.distribution), sps.rv_continuous):
            return self.distribution.rvs(size)
        else:
            return self.distribution.resample(size)[0]


In [5]:
def interpol_distribution(sample, bins=30):
    hst = np.histogram(sample, density=True, bins=bins)
    return interp1d(hst[1][:-1], hst[0], kind='linear',
                    fill_value=0.0, bounds_error=False)

In [6]:
def zero_step(model: CMFRecommender, user_info, item_info, topn=52):
    new_feedback = []

    maximal_user, maximal_item = model.get_max_index()
    old_users = model.get_known_users_info().set_index("UserId")
    old_items = model.get_known_items_info().set_index("ItemId")
    for i in range(topn):
        user_id = np.random.choice(round(maximal_user) - 1)
        for index, item_row in item_info.iterrows():
            deal = sps.bernoulli.rvs(
                usefulness(old_users.loc[user_id]["F"], item_row["F"], epsilon.rvs()))  # моделируем сделки
            new_feedback.append((user_id, item_row["ItemId"], deal))
    for index, user_row in user_info.iterrows():
        w_offered = model.recommend_items_cold(user_row["F"], topn)["ItemId"]
        for w in w_offered:
            if not(w in old_items.index.values):
                feature = item_info.loc[item_info.ItemId == w]["F"]
            else:
                feature = old_items.loc[w]["F"]
            deal = sps.bernoulli.rvs(usefulness(user_row["F"], feature, epsilon.rvs()))  # моделируем сделки
            
            new_feedback.append((user_row["UserId"], w, deal))
            

    model.retrain(new_feedback, user_info, item_info)
    return model


In [7]:
def print_distributions(num_of_iteration, axs, user_info, item_info):
    sns.kdeplot(user_info["F"], ax=axs[0], label=f"Itration number = {num_of_iteration}")
    axs[0].set_title("User Distribution")
    sns.kdeplot(data=item_info["F"], ax=axs[1])
    axs[1].set_title("Item Distribution")

In [7]:
def dynamic_system_iterate(model, customer_distribution, w_distribution, c_size=10, w_size=10,
                           num_of_steps=5,
                           topn=1, delta=1e-4, visualize_distributions=None):
    maximal_user, maximal_item = model.get_max_index()
    user_info = pd.DataFrame(
        {"F": customer_distribution.rvs(size=c_size)})  # size = (c_size, c_feature_size) в многомерном случае 
    user_info["UserId"] = np.arange(maximal_user + 1, maximal_user + 1 + c_size)
    
    item_info = pd.DataFrame(
        {"F": w_distribution.rvs(size=w_size)})  # size = (w_size, w_feature_size) в многомерном случае 
    item_info["ItemId"] = np.arange(maximal_item + 1, maximal_item + 1 + w_size)
    model = zero_step(model, user_info, item_info, topn=topn)

    if visualize_distributions is not None:
        print_distributions(visualize_distributions[0], visualize_distributions[1], user_info, item_info)

    new_feedback = []
    old_users = model.get_known_users_info().set_index("UserId")
    old_items = model.get_known_items_info().set_index("ItemId")
    
    for step in range(1, num_of_steps + 1):
        counter = 0
        for index, user_row in user_info.iterrows():
            w_offered = model.recommend_items(user_row["UserId"], item_info['ItemId'], topn=topn, exclude_rated=True)[
                "ItemId"]
            for w in w_offered:
                if not(w in old_items.index.values):
                    feature = item_info.loc[item_info.ItemId == w]["F"]
                else:
                    feature = old_items.loc[w]["F"]
                deal = sps.bernoulli.rvs(usefulness(user_row["F"], feature, epsilon.rvs()))  # моделируем сделки
                counter += deal
                new_feedback.append((user_row["UserId"], w, deal))

        model.retrain(new_feedback, [], [])

    # смена распределения
    # print(user_info['Feedback'])
    new_feedback_df = pd.DataFrame(new_feedback, columns=['UserId', 'ItemId', 'Feedback'])
    metr = new_feedback_df.groupby('UserId')['Feedback'].mean().reset_index()
    
    grouped_users = new_feedback_df.groupby('UserId')['Feedback'].mean().reset_index()
    grouped_users['Feedback'] += delta
    
    user_info = user_info.merge(grouped_users, how="inner", on='UserId')
    customer_distribution = Distribution_Handler(sps.gaussian_kde(user_info["F"], bw_method=.105, weights=user_info['Feedback']))
    grouped_items = new_feedback_df.groupby('ItemId')['Feedback'].mean().reset_index()


    item_info = item_info.merge(grouped_items, how="left", on='ItemId').fillna(0)
    item_info['Feedback'] += delta
    
    w_distribution = Distribution_Handler(sps.gaussian_kde(item_info["F"], bw_method=.09,weights=item_info['Feedback']))
    
    return customer_distribution, w_distribution, metr



In [8]:
start_c_size = 100
start_w_size = 100

user_info = pd.DataFrame({"F": customer_distribution.rvs(size=start_c_size)})  # генерим датасет для нулевой итерации
user_info["UserId"] = np.arange(start_c_size)

item_info = pd.DataFrame({"F": w_distribution.rvs(size=start_w_size)})
item_info["ItemId"] = np.arange(start_w_size)
feedback = []

# print_distributions(0, axs,user_info, item_info)

for i, user_row in user_info.iterrows():
    for j, item_row in item_info.iterrows():
        deal = sps.bernoulli.rvs(usefulness(user_row["F"], item_row["F"], epsilon.rvs()))
        feedback.append((user_row["UserId"], item_row["ItemId"], deal))

In [9]:
model = CMFRecommender(capacity=11000, num_of_factors=140)
model.fit(feedback, user_info, item_info)

In [10]:
new_cust_distr, new_w_distr = customer_distribution, w_distribution

In [11]:
 # new_cust_distr, new_w_distr, _ = dynamic_system_iterate(model, new_cust_distr, new_w_distr, c_size=10, w_size=10, visualize_distributions=True)

In [ ]:
c_size = 128
w_size = 128
big_steps = 50
hit_metrics = []
curr_hit = 0
fig, axs = plt.subplots(ncols=2)
for i in tqdm(range(big_steps)):
    if i % 5 == 0:
        new_cust_distr, new_w_distr, current_metr_distr = dynamic_system_iterate(model, new_cust_distr, new_w_distr, c_size=c_size,
                                                                   w_size=w_size, num_of_steps=3, visualize_distributions=(i, axs))
        plt.figure(figsize=(7,7))
        plt.hist(current_metr_distr["Feedback"], density=True)
    else:
        new_cust_distr, new_w_distr, current_metr_distr = dynamic_system_iterate(model, new_cust_distr, new_w_distr, c_size=c_size,
                                                                       w_size=w_size, num_of_steps=3, visualize_distributions=None)
    hit_metrics.append(curr_hit)

fig.legend(loc='center left', bbox_to_anchor=(1, 0.5))
fig.show()

In [ ]:
plt.plot(np.arange(big_steps), hit_metrics)

In [ ]:
big_steps = 50
hit_metrics = []
curr_hit = 0
fig, axs = plt.subplots(ncols=2)
for i in tqdm(range(big_steps)):
    # if (i + 1) % 5 == 0:
    new_cust_distr, new_w_distr, curr_hit = dynamic_system_iterate(model, new_cust_distr, new_w_distr,
                                                                   c_size=50, w_size=100, visualize_distributions=(i, axs))
    # else:
    #     new_cust_distr, new_w_distr, curr_hit = dynamic_system_iterate(i, axs, model, new_cust_distr, new_w_distr, c_size=50,
    #                                                                    w_size=50, visualize_distributions=False)
    hit_metrics.append(curr_hit)



In [ ]:
plt.plot(np.arange(big_steps - 1), hit_metrics)

In [ ]:
def dynamic_system_iterate_CB(model: ContentBasedRecommender, customer_distribution, w_distribution, c_size=10, w_size=10,
                           num_of_steps=1,
                           topn=10, delta=0.00001, visualize_distributions=None):
    maximal_user, maximal_item = model.get_max_index()
    user_info = pd.DataFrame(
        {"F": customer_distribution.rvs(size=c_size)})  # size = (c_size, c_feature_size) в многомерном случае 
    user_info["UserId"] = np.arange(maximal_user + 1, maximal_user + 1 + c_size)

    item_info = pd.DataFrame(
        {"F": w_distribution.rvs(size=w_size)})  # size = (w_size, w_feature_size) в многомерном случае 
    item_info["ItemId"] = np.arange(w_size)
    
    print(item_info)
    # model = zero_step_CB(model, user_info, item_info, topn=topn)

    if visualize_distributions is not None:
        print_distributions(visualize_distributions[0], visualize_distributions[1], user_info, item_info)

    new_feedback = []
    
    
    for step in range(1, num_of_steps + 1):
        counter = 0
        for index, user_row in user_info.iterrows():
            w_offered = model.recommend_items_new(user_row["F"], item_info[["F"]], topn=topn)[
                "ItemId"]
            for w in w_offered:
                deal = sps.bernoulli.rvs(usefulness(user_row["F"], w, epsilon.rvs()))  # моделируем сделки
                counter += deal
                new_feedback.append((user_row["UserId"], w, deal))
        
        model.retrain(new_feedback, [], [])

    # смена распределения
    new_feedback_df = pd.DataFrame(new_feedback, columns=['UserId', 'ItemId', 'Feedback'])
    new_feedback_df["Feedback"] += delta
    grouped_users = new_feedback_df.groupby('UserId')['Feedback'].mean().reset_index()

    user_info = user_info.merge(grouped_users, how="inner", on='UserId')

    customer_distribution = Distribution_Handler(sps.gaussian_kde(user_info["F"], weights=user_info['Feedback']))
    grouped_items = new_feedback_df.groupby('ItemId')['Feedback'].mean().reset_index()
    # grouped_items['Feedback'] += delta

    item_info = item_info.merge(grouped_items, how="inner", on='ItemId')

    w_distribution = Distribution_Handler(sps.gaussian_kde(item_info["F"], weights=item_info['Feedback']))

    return customer_distribution, w_distribution, float(counter) / c_size



In [ ]:
model = ContentBasedRecommender(capacity=30000, num_of_factors=50)
model.fit(feedback, user_info, item_info)

In [181]:
new_cust_distr, new_w_distr = customer_distribution, w_distribution
big_steps = 2
hit_metrics = []
curr_hit = 0
fig, axs = plt.subplots(ncols=2)
for i in tqdm(range(big_steps)):
    if (i + 1) % 5 == 0:
        new_cust_distr, new_w_distr, curr_hit = dynamic_system_iterate_CB(model, new_cust_distr, new_w_distr, c_size=50,
                                                                       w_size=50, num_of_steps=3,
                                                                       visualize_distributions=(i, axs))
    else:
        new_cust_distr, new_w_distr, curr_hit = dynamic_system_iterate_CB(model, new_cust_distr, new_w_distr, c_size=50,
                                                                       w_size=50, num_of_steps=3,
                                                                       visualize_distributions=None)
    hit_metrics.append(curr_hit)

fig.legend(loc='center left', bbox_to_anchor=(1, 0.5))
fig.show()

In [25]:
size = 1024*16
data = sps.norm.rvs(0, 1, size)


In [26]:
pdf = interpol_distribution(data, bins=20)
x = np.linspace(-5, 5, 300)
plt.plot(x, pdf(x))
sns.kdeplot(data)

In [27]:
def get_resample(data):
    distr = sps.gaussian_kde(data)
    return distr.resample(size)[0]

In [28]:
fig, ax = plt.subplots(1, figsize=(10, 10))
data_new = data
for i in range(100):
    data_new = get_resample(data_new)
    if i % 20 == 0:
        sns.kdeplot(data_new, ax=ax, label=f"Itration number = {i}")
plt.legend()
plt.show()

In [57]:
def rand_dist(x, pdf, nvals):
    """Produce nvals random samples from pdf(x), assuming constant spacing in x."""

    # get cumulative distribution from 0 to 1
    cumpdf = np.cumsum(pdf)
    cumpdf *= 1/cumpdf[-1]

    # input random values
    randv = np.random.uniform(size=nvals)

    # find where random values would go
    idx1 = np.searchsorted(cumpdf, randv)
    # get previous value, avoiding division by zero below
    idx0 = np.where(idx1==0, 0, idx1-1)
    idx1[idx0==0] = 1

    # do linear interpolation in x
    frac1 = (randv - cumpdf[idx0]) / (cumpdf[idx1] - cumpdf[idx0])
    randdist = x[idx0]*(1-frac1) + x[idx1]*frac1

    return randdist

In [66]:

def get_resample_norm(data):
    pdf = interpol_distribution(data)
    x = np.linspace(-3, 3, 10000)
    return rand_dist(x, pdf(x), len(data))

In [67]:
fig, ax = plt.subplots(1, figsize=(10, 10))
data_new = data
for i in range(100):
    data_new = get_resample_norm(data_new)
    if i % 20 == 0:
        sns.kdeplot(data_new, ax=ax, label=f"Itration number = {i}")
plt.legend()
